In [1]:
import os 
os.environ['KERAS_BACKEND'] = 'tensorflow'
# Math
import numpy as np 
# NN Buiding
import tensorflow as tf 
from tensorflow.keras.layers import (Input, 
                                     Conv2D, 
                                     MaxPooling2D, 
                                     Flatten, 
                                     Dense, 
                                     BatchNormalization, 
#                                      InstanceNormalization
                                    )
from tensorflow.data import Dataset
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.metrics import Accuracy, Precision, Recall, AUC
# Augmentation


# Visualization 
import matplotlib.pyplot as plt 

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Load Dataset

In [2]:
IMAGE_SHAPE = [100, 100, 1]
NUM_CLASSES = 65 

In [3]:
def load_filepaths_labels(folder_path):
    file_paths = []
    labels = []

    # List all directories (labels) in the given folder
    for label in os.listdir(folder_path):
        label_path = os.path.join(folder_path, label)

        # Check if the item in the directory is a subdirectory
        if os.path.isdir(label_path):
            # List all files in the subdirectory
            files = os.listdir(label_path)
            
            # Create file paths and corresponding labels
            file_paths.extend([os.path.join(label_path, file) for file in files])
            labels.extend([int(label)] * len(files))

    return file_paths, labels

In [4]:
def load_decode_image(filepath, label):
    # One-hot encoding the label
    label = tf.one_hot(label, depth=NUM_CLASSES)

    # Read and decode the image
    image = tf.io.read_file(filepath)
    image = tf.image.decode_image(image, channels=IMAGE_SHAPE[-1])
    image = image * -1

    image = tf.image.convert_image_dtype(image, tf.float32)
    
    # Normalize the image
    # Resize the image while maintaining aspect ratio
    image = tf.image.resize_with_pad(image, target_height=IMAGE_SHAPE[0], target_width=IMAGE_SHAPE[1], method = 'bicubic')

    return image, label

def build_dataset(filepaths, labels, batch_size = 8, shuffle = True, prefetch = True): 
    
    dataset = tf.data.Dataset.from_tensor_slices((filepaths, labels))
    
    dataset = dataset.map(lambda x, y: load_decode_image(x, y))
    dataset = dataset.batch(batch_size)
    
    if shuffle: 
        dataset = dataset.shuffle(2)
        
    if prefetch: 
        dataset = dataset.prefetch(tf.data.AUTOTUNE)
        
    return dataset  

# Build Model

In [5]:
def enc_bl(n_filters, strides=2, bnorm=True) -> tf.keras.Model: 
    # Convolutional block with optional batch normalization
    
    # Convolutional layer with kernel size 2 and specified number of filters
    # Use 'strides' instead of 'stribes' (typo) and specify padding='same'
    conv_init = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.2)
    head = tf.keras.Sequential([
        Conv2D(n_filters, kernel_size=2, strides=1, padding='same', bias_initializer=conv_init), 
        MaxPooling2D(2, strides=strides)  # Corrected 'stribes' to 'strides'
    ])
    
    if bnorm: 
        head.add(BatchNormalization())
        
    return head

In [6]:
# inputs shape = [None, 256, 256, 1]
def build_model(inputs_shape, num_classes = 65) -> tf.keras.Model: 
    inputs = Input(shape = inputs_shape)
    
    x = enc_bl(16)(inputs) #    [None, 100, 100, 16]
    x = enc_bl(32)(x) # shape : [None, 50, 50, 32]
    x = enc_bl(64)(x) # shape : [None, 25, 25, 64] 
    x = enc_bl(128, strides = 5)(x) # [None, 5, 5, 128]
    
    x = Flatten()(x)
    outputs = Dense(units = num_classes, activation='softmax')(x)
    
    return tf.keras.Model(inputs, outputs)

In [7]:
fileps, labels = load_filepaths_labels('/kaggle/input/arabic-letters-classification/Final_Arabic_Alpha_dataset/Final_Arabic_Alpha_dataset/train')

dataset = build_dataset(fileps, labels, batch_size = 512, shuffle = False)

model = build_model(IMAGE_SHAPE, 65)

In [8]:
# Use AdamW optimizer
optimizer = tf.keras.optimizers.AdamW(learning_rate=1e-6)

# Use more metrics
metrics = [Accuracy()]

# Compile the model
model.compile(optimizer=optimizer, loss='binary_crossentropy')

# Define callbacks
# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = LearningRateScheduler(lambda epoch: 1e-6 * 0.9**epoch)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)
# Add any other callbacks you may need


# Train the model
model.fit(dataset, 
          epochs=50,
#           steps_per_epoch = 5, 
#           validation_data=val_dataset, 
          callbacks=[lr_scheduler, model_checkpoint]
         )

Epoch 1/50
84/84 [==============================] - 52s 457ms/step - loss: 0.9090 - lr: 1.0000e-06
Epoch 2/50
84/84 [==============================] - 18s 213ms/step - loss: 0.8907 - lr: 9.0000e-07
Epoch 3/50
84/84 [==============================] - 17s 202ms/step - loss: 0.8748 - lr: 8.1000e-07
Epoch 4/50
84/84 [==============================] - 17s 202ms/step - loss: 0.8608 - lr: 7.2900e-07
Epoch 5/50
84/84 [==============================] - 18s 219ms/step - loss: 0.8485 - lr: 6.5610e-07
Epoch 6/50
84/84 [==============================] - 19s 220ms/step - loss: 0.8375 - lr: 5.9049e-07
Epoch 7/50
84/84 [==============================] - 18s 214ms/step - loss: 0.8278 - lr: 5.3144e-07
Epoch 8/50
84/84 [==============================] - 17s 204ms/step - loss: 0.8192 - lr: 4.7830e-07
Epoch 9/50
84/84 [==============================] - 17s 206ms/step - loss: 0.8115 - lr: 4.3047e-07
Epoch 10/50
84/84 [==============================] - 17s 203ms/step - loss: 0.8047 - lr: 3.8742e-07
Epoch 11/

In [9]:
model.save('/kaggle/working/')